In [1]:
import pandas as pd

sentence = pd.read_csv('../data/sentiment_score_2.csv', sep='\t')
topic = pd.read_excel('../data/4_clusters_cbow200_kmeans-words_fixed-Wu.xlsx', engine='openpyxl')
topic['Phrases'] = topic['Phrases'].apply(lambda x: x.split('，'))
sentence

,sentence,sentiment,dict
0,很不错,1.000000,kansei
1,很方便,0.913670,kansei
2,交通很方便,1.598922,kansei
3,房间很干净,1.750000,kansei
4,虽然房间面积普遍都不大,-0.818779,kansei
...,...,...,...
1365343,适合孩子,0.233304,boson
1365344,就是房间漏水,-0.264336,boson
1365345,没有热水,-0.113943,boson
1365346,巴适的板,0.450991,boson


In [2]:
all_word = topic['Phrases'].tolist()
facility = all_word[0]
parking = all_word[1]
service = all_word[2]
special_care = all_word[3]
room = all_word[4]
value = all_word[5]
booking = all_word[6]
cleanliness = all_word[7]
bathroom = all_word[8]
location_transport = all_word[9]
surrounding = all_word[10]
decoration = all_word[11]
food_drink = all_word[12]
# 构造dict
# 0: facility,
# 1: parking,
# 2: service,
# 3: special_care,
# 4: room,
# 5: value,
# 6: booking,
# 7: cleanliness,
# 8: bathroom,
# 9: location_transport,
# 10: surrounding,
# 11: decoration,
# 12: food_drink
topic_dict = {
    0: facility,
    1: parking,
    2: service,
    3: special_care,
    4: room,
    5: value,
    6: booking,
    7: cleanliness,
    8: bathroom,
    9: location_transport,
    10: surrounding,
    11: decoration,
    12: food_drink
}
word_dict = {}
for i, words in enumerate(all_word):
    for word in words:
        word_dict[word] = i
# word_dict

In [3]:
import re

import jieba
import xiangshi as xs

digits = '0123456789'
# 去掉baidu_stopwords中的属性词
stop_df = pd.read_csv('../data/baidu_stopwords.txt')
all_df = pd.DataFrame([y for x in all_word for y in x])
stop_df.columns = ['word']
all_df.columns = ['word']
word_duplicated = pd.merge(stop_df, all_df, on='word')['word']
stop_list = list(filter(lambda x: x not in word_duplicated, stop_df))


# 保留中文字符，删除非中文字符
def find_chinese(doc):
    pattern = re.compile(r'[^\u4e00-\u9fa5]')
    chinese = re.sub(pattern, "", doc)
    return chinese


def pre_process(text0):
    # 去标点符号
    text0 = re.sub("[\s+\.\!\/_,$%^*(+\"\'～]+|[+——！，。？?、~@#￥%……&*（）．；：】【|]+", "", str(text0))
    # 去掉数字
    text0 = text0.translate(str.maketrans('', '', digits))
    # 去掉非中文字符
    text0 = find_chinese(text0)
    # 分词
    text0 = jieba.cut(text0)
    # 去停用词
    text0 = [word.strip() for word in text0 if word not in stop_list]

    return text0


# 判断短文本属于哪个主题，根据dictionary
def match_topic(word_list):
    # 42作为没有匹配的标志
    result = 42
    for word in word_list:
        # 遍历先验词库
        # 将首次出现的词作为分类依据
        if word in topic_dict.get(0):
            result = 0
            break
        elif word in topic_dict.get(1):
            result = 1
            break
        elif word in topic_dict.get(2):
            result = 2
            break
        elif word in topic_dict.get(3):
            result = 3
            break
        elif word in topic_dict.get(4):
            result = 4
            break
        elif word in topic_dict.get(5):
            result = 5
            break
        elif word in topic_dict.get(6):
            result = 6
            break
        elif word in topic_dict.get(7):
            result = 7
            break
        elif word in topic_dict.get(8):
            result = 8
            break
        elif word in topic_dict.get(9):
            result = 9
            break
        elif word in topic_dict.get(10):
            result = 10
            break
        elif word in topic_dict.get(11):
            result = 11
            break
        elif word in topic_dict.get(12):
            result = 12
            break

    # 字典中没有匹配，返回-1
    if result == 42:
        result = -1

    return result


# 根据相似度匹配属性
def match_by_similarity(text1):
    try:
        # 计算text与字典中所有key的相似度，取最大值为最终结果
        max_similarity = 0

        result = -1
        for key, value_list in topic_dict.items():
            sim = 0
            for v in value_list:
                sim = max(sim, xs.cossim([text1, v]))  # 找到当前key下最大的相似度
            if max_similarity < sim:
                max_similarity = sim
                result = key

        return result
    except TypeError:
        # 出错返回-2
        return -2

In [5]:
from tqdm import tqdm

tqdm.pandas()

sentence['words'] = sentence['sentence'].progress_apply(pre_process)
sentence['topic'] = sentence['words'].progress_apply(match_topic)
dict_topic = sentence[sentence['topic'] != -1]
sim_topic = sentence[sentence['topic'] == -1]
dict_topic['match'] = 'dict'
sim_topic['match'] = 'sim'
dict_topic.shape, sim_topic.shape

100%|██████████| 1365348/1365348 [00:33<00:00, 40673.51it/s]
C:\Users\62774\AppData\Local\Temp\ipykernel_15336\1181994722.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dict_topic['match'] = 'dict'
C:\Users\62774\AppData\Local\Temp\ipykernel_15336\1181994722.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sim_topic['match'] = 'sim'


((859689, 6), (505659, 6))

In [6]:
dict_topic = dict_topic.drop(labels=['words', 'match'], axis=1)
sim_topic = sim_topic.drop(labels=['words', 'match'], axis=1)
dict_topic.to_csv('../data/sentence_match_result_dict.csv_2', sep='\t', index=False)
sim_topic.to_csv('../data/sentence_match_result_without_match_2.csv', sep='\t', index=False)